# btorch (word_embeddings) tutorial
https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html  

Author: Robert Guthrie, Brian

In [1]:
import torch
import btorch.nn as nn
import btorch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])

vocab = set(test_sentence)  #all unique words
word_to_ix = {word: i for i, word in enumerate(vocab)}

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


## Define a Model in btorch

In [3]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        # here inputs is (CONTEXT_SIZE,) reshape to 
        # embeds (1, EMBEDDING_DIM* CONTEXT_SIZE)
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs
    
    @classmethod
    def train_epoch(cls, net, criterion, trainloader, optimizer, epoch_idx, device='cuda', config=None):
        total_loss = 0
        for context, target in trainloader:
            # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
            # into integer indices and wrap them in tensors)
            context_idxs = torch.tensor([word_to_ix[w[0]] for w in context], dtype=torch.long)
            # Step 2. Recall that torch *accumulates* gradients. Before passing in a
            # new instance, you need to zero out the gradients from the old
            # instance
            net.zero_grad()

            # Step 3. Run the forward pass, getting log probabilities over next
            # words
            log_probs = net(context_idxs)

            # Step 4. Compute your loss function. (Again, Torch wants the target
            # word wrapped in a tensor)
            # Here we are predicting the target word, by outputing a (V,) vector
            # ans use softmax to choose the highest proba word
            loss = criterion(log_probs, torch.tensor([word_to_ix[target[0]]], dtype=torch.long))

            # Step 5. Do the backward pass and update the gradient
            loss.backward()
            optimizer.step()
            
            # Get the Python number from a 1-element Tensor by calling tensor.item()
            total_loss += loss.item()
        return total_loss



model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
model._lossfn = nn.NLLLoss()
model._optimizer = optim.SGD(model.parameters(), lr=0.001)

train_loader = torch.utils.data.DataLoader(ngrams, batch_size=1, shuffle=True)

model.fit(train_loader)


# To get the embedding of a particular word, e.g. "beauty"
print(model.embeddings.weight[word_to_ix["beauty"]])

Epoch 0: Training loss: 523.1158204078674. Testing loss: Not Provided
Epoch 1: Training loss: 520.7239346504211. Testing loss: Not Provided
Epoch 2: Training loss: 518.3480830192566. Testing loss: Not Provided
Epoch 3: Training loss: 515.9878144264221. Testing loss: Not Provided
Epoch 4: Training loss: 513.6441121101379. Testing loss: Not Provided
Epoch 5: Training loss: 511.3161165714264. Testing loss: Not Provided
Epoch 6: Training loss: 509.0015251636505. Testing loss: Not Provided
Epoch 7: Training loss: 506.69941306114197. Testing loss: Not Provided
Epoch 8: Training loss: 504.41120433807373. Testing loss: Not Provided
Epoch 9: Training loss: 502.1318509578705. Testing loss: Not Provided
tensor([-6.6014e-01, -1.1942e+00, -2.6075e-01, -1.0646e-03,  1.5511e+00,
        -9.2689e-01, -8.4736e-01,  1.5865e-01,  8.5613e-01, -2.0895e+00],
       grad_fn=<SelectBackward0>)


In [4]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = (
        [raw_text[i - j - 1] for j in range(CONTEXT_SIZE)]
        + [raw_text[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['are', 'We', 'to', 'study'], 'about'), (['about', 'are', 'study', 'the'], 'to'), (['to', 'about', 'the', 'idea'], 'study'), (['study', 'to', 'idea', 'of'], 'the'), (['the', 'study', 'of', 'a'], 'idea')]


In [5]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim*context_size, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view(1,-1)
        x = F.relu(self.linear1(embeds))
        x = self.linear2(x)
        log_probs = F.log_softmax(out, dim=1)
        return -1*log_probs
    
    @classmethod
    def train_epoch(cls, net, criterion, trainloader, optimizer, epoch_idx, device='cuda', config=None):
        total_loss = 0
        for context, target in trainloader:
            
            # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
            # into integer indices and wrap them in tensors)
            context_idxs = make_context_vector(context, word_to_ix) 

            # Step 2. Recall that torch *accumulates* gradients. Before passing in a
            # new instance, you need to zero out the gradients from the old
            # instance
            net.zero_grad()

            # Step 3. Run the forward pass, getting log probabilities over next
            # words
            log_probs = model(context_idxs)

            # Step 4. Compute your loss function. (Again, Torch wants the target
            # word wrapped in a tensor)
            # Here we are predicting the target word, by outputing a (V,) vector
            # ans use softmax to choose the highest proba word
            loss = loss_function(log_probs, torch.tensor([word_to_ix[target[0]]], dtype=torch.long))

            # Step 5. Do the backward pass and update the gradient
            loss.backward()
            optimizer.step()

            # Get the Python number from a 1-element Tensor by calling tensor.item()
            total_loss += loss.item()
        return total_loss
    
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w[0]] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

In [6]:

model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE*2)
model._optimizer = optim.SGD(model.parameters(), lr=0.001)
model._lossfn = nn.NLLLoss()

train_loader = torch.utils.data.DataLoader(data, batch_size=1, shuffle=True)

model.fit(train_loader)

# To get the embedding of a particular word, e.g. "computer"
print(model.embeddings.weight[word_to_ix["computer"]])

Epoch 0: Training loss: 229.51315832138062. Testing loss: Not Provided
Epoch 1: Training loss: 228.04372262954712. Testing loss: Not Provided
Epoch 2: Training loss: 226.58604526519775. Testing loss: Not Provided
Epoch 3: Training loss: 225.13886785507202. Testing loss: Not Provided
Epoch 4: Training loss: 223.69987678527832. Testing loss: Not Provided
Epoch 5: Training loss: 222.26979398727417. Testing loss: Not Provided
Epoch 6: Training loss: 220.8470311164856. Testing loss: Not Provided
Epoch 7: Training loss: 219.43106651306152. Testing loss: Not Provided
Epoch 8: Training loss: 218.02213144302368. Testing loss: Not Provided
Epoch 9: Training loss: 216.6177306175232. Testing loss: Not Provided
tensor([-0.3597, -0.1511, -0.2944, -0.5894,  0.4267, -0.0143,  0.4768,  0.1845,
         0.5582,  1.5608], grad_fn=<SelectBackward0>)
